In [38]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from collections import defaultdict
from datetime import datetime 
from tqdm import tqdm_notebook
from collections import Counter
from bs4 import BeautifulSoup as bs
from tqdm import tqdm_notebook
from selenium.common.exceptions import NoSuchElementException

import urllib.request
import urllib.parse
import lxml.html
import requests
import pandas as pd
import time
import math
import re
import random

# word2vec
from gensim.models.word2vec import Word2Vec
import gensim 
import gensim.models as g
plt.rc('font', family='nanumgothic')
plt.rc('axes', unicode_minus=False)

NameError: name 'plt' is not defined

In [2]:
# driver path
path = 'C:/Users/Administrator/chromedriver'
driver = webdriver.Chrome(path)

In [17]:
# url 데이터프레임 생성
urlDf = pd.DataFrame(data = {'유튜버': [],
                              '제목': [],
                              'url': [],
                              '조회수': [],
                              '날짜':[],
                              '댓글수': []})

In [18]:
urlDf

,유튜버,제목,url,조회수,날짜,댓글수
0,디렉터파이,"(*Eng) 내 피부가 이랬어?! 민감, 지성, 건성 에센스 BEST 궁합 찾는 법...",https://www.youtube.com/watch?v=YkX9SA166RM&t=51s,317550,2018-06-08,302
1,디렉터파이,"[디파GO]민감 건성, 보습과 탄력? 화장품 바르는 4가지 방법(디렉터파이 피부 측...",https://www.youtube.com/watch?v=6zti2UXhQ3A,183588,2017-09-01,177
2,디렉터파이,"(**Eng.)여드름, 좁쌀, 각질 부자 모여! 피부타입별 성분 궁합& 착한 크림들...",https://www.youtube.com/watch?v=q12lCuRff34,1432136,2017-10-20,1080
3,디렉터파이,(*Eng)피부타입별 토너 사용법 따로 있다? 효과 검증 화장솜 궁합까지. by 디...,https://www.youtube.com/watch?v=C8wXn0Xdp1c,961828,2018-05-04,849
4,디렉터파이,"속건조 해결?! 지성, 수부지 민감한 피부 위한 기초 스킨케어 by. 디렉터파이",https://www.youtube.com/watch?v=N1M2ocbcHb0,943392,2017-08-11,786
5,이사배,[Eng] 민감피부🤦🏻‍♀ 나의 하루종일 피부진정방법🍀 Calming Skincar...,https://www.youtube.com/watch?v=xsBKNABqi5E,456728,2018-06-09,426
6,디렉터파이,(*Eng) 국내 더마 코스메틱 Top5 성분 효과 검증 by 디렉터파이,https://www.youtube.com/watch?v=_sejSV5yXro,184022,2018-10-29,282


## 잠재고객 url 리스트 만들기

In [146]:
#reply_cnt = int(driver.find_element_by_css_selector('#count > yt-formatted-string').text[3:-1]) #댓글수
user_list = []
reply_list = []

for i in range(len(urlDf)):
    driver.get(urlDf.url[i])
    reply_cnt = urlDf.댓글수[i]

    start = datetime.now()
    batang = driver.find_element_by_css_selector("body") 
    while True : 
        now = datetime.now() 

        batang.send_keys(Keys.PAGE_DOWN)  
        time.sleep(1)

        # 더보기 클릭하는 반복문
        try:
            more_btns = driver.find_elements_by_xpath('//*[@id="more-replies"]')
            for more_btn in more_btns:
                try:
                    more_btn.click()
                    time.sleep(1)
                except:
                    pass            
        except:
            pass

        replys = driver.find_elements_by_css_selector('yt-formatted-string#content-text.style-scope.ytd-comment-renderer')
        check_cnt = len(replys) 
        if check_cnt == reply_cnt:
            break
        elif (now-start).seconds > 300:
            break
    
    time.sleep(2)

    # 댓글 단 사람들의 url
    user_urls = driver.find_elements_by_css_selector('#author-text')  
    time.sleep(2)
    for user_url in user_urls:
        user_list.append([user_url.text, user_url.get_attribute('href')])

    # 댓글 단 사람들의 댓글
    replys = driver.find_elements_by_css_selector('yt-formatted-string#content-text.style-scope.ytd-comment-renderer')
    time.sleep(2)
    for reply in replys:
        reply_list.append(reply.text)
        
    continue

In [148]:
replyDf = pd.DataFrame(reply_list, columns = ['댓글내용'])
replyDf.head()

,댓글내용
0,"안녕하세요, 오늘 올라갈 영상은 내 피부타입 아는 법-기초(1편 에센스), (2편 ..."
1,디렉터 파이 Do english subtitle please~🙏🏻
2,겨울버전도 해주세요ㅠㅠ
3,코랑 이마엔 기름이많고 다른곳은 건조하면 수부지인가요?
4,디렉터 파이 토너나 다른 것들은 안바르고 에센스 수분크림만 발라도될까요?? 고수님들...


In [157]:
replyDf.to_excel('더마토리_유튜브댓글.xlsx')

In [158]:
replyDf = pd.read_excel('더마토리_유튜브댓글.xlsx', index_col=0)
replyDf.head()

,댓글내용
0,"안녕하세요, 오늘 올라갈 영상은 내 피부타입 아는 법-기초(1편 에센스), (2편 ..."
1,디렉터 파이 Do english subtitle please~🙏🏻
2,겨울버전도 해주세요ㅠㅠ
3,코랑 이마엔 기름이많고 다른곳은 건조하면 수부지인가요?
4,디렉터 파이 토너나 다른 것들은 안바르고 에센스 수분크림만 발라도될까요?? 고수님들...


In [150]:
userDf = pd.DataFrame(user_list, columns = ['이름', 'url'])
userDf.head()

,이름,url
0,,https://www.youtube.com/channel/UCqrNqg3UgVoD3...
1,pat ty,https://www.youtube.com/channel/UCd6DPfmyJa0Cr...
2,레브,https://www.youtube.com/channel/UCVYFzlQ7SOzuG...
3,룽이는섭섭해,https://www.youtube.com/channel/UCnhlLQh0p89Mc...
4,유준상,https://www.youtube.com/channel/UCrZR1TakPPMOa...


In [151]:
userDfUni = userDf.drop_duplicates()

In [155]:
userDfUni.to_excel('더마토리_유튜브유저.xlsx')

In [174]:
userDf = pd.read_excel('더마토리_유튜브유저.xlsx', index_col=0)
userDf.head()

,이름,url
0,NaN,https://www.youtube.com/channel/UCqrNqg3UgVoD3...
1,pat ty,https://www.youtube.com/channel/UCd6DPfmyJa0Cr...
2,레브,https://www.youtube.com/channel/UCVYFzlQ7SOzuG...
3,룽이는섭섭해,https://www.youtube.com/channel/UCnhlLQh0p89Mc...
4,유준상,https://www.youtube.com/channel/UCrZR1TakPPMOa...


In [176]:
userDf = userDf.reset_index(drop=True)
userDf.head()

,이름,url
0,NaN,https://www.youtube.com/channel/UCqrNqg3UgVoD3...
1,pat ty,https://www.youtube.com/channel/UCd6DPfmyJa0Cr...
2,레브,https://www.youtube.com/channel/UCVYFzlQ7SOzuG...
3,룽이는섭섭해,https://www.youtube.com/channel/UCnhlLQh0p89Mc...
4,유준상,https://www.youtube.com/channel/UCrZR1TakPPMOa...


## 댓글 단 사람들의 구독자 크롤링

In [186]:
## NaN값 제거
userDf = userDf[userDf.이름.isnull()!=True]
userDf = userDf.reset_index(drop=True)
userDf.head()

,이름,url
0,pat ty,https://www.youtube.com/channel/UCd6DPfmyJa0Cr...
1,레브,https://www.youtube.com/channel/UCVYFzlQ7SOzuG...
2,룽이는섭섭해,https://www.youtube.com/channel/UCnhlLQh0p89Mc...
3,유준상,https://www.youtube.com/channel/UCrZR1TakPPMOa...
4,잘생김,https://www.youtube.com/channel/UCxDgUcK6fn9Lm...


In [375]:
len(userDf)

3073

In [350]:
## version1
youtuberList = []

for i in range(len(userDf)):
    driver.get(userDf.url[i])
    time.sleep(1.5)
    
    ## 1.구독자가 있는 경우 ##
    if '구독정보' in driver.page_source:
        youtuberLi = []
        while True:
            time.sleep(2)
            youtuberEles = driver.find_elements_by_css_selector('span#title.style-scope.ytd-grid-channel-renderer')
            youtuberTexts = ', '.join([ youtuberEle.text for youtuberEle in youtuberEles ])
            youtuberLi.extend([youtuberTexts])

            time.sleep(1)
            try:
                more_btn = driver.find_element_by_css_selector('#right-arrow > ytd-button-renderer')
                time.sleep(3)
                more_btn.click() # 더보기 클릭
                continue
            except:
                break

        time.sleep(2)
        youtuber_text = ', '.join([youtuber for youtuber in youtuberLi])
        youtuberList.append([userDf.이름[i], youtuber_text])
        continue
        
    ## 2.구독자가 없는 경우 ##
    else: 
        youtuberList.append([userDf.이름[i], '없음'])
        continue

KeyboardInterrupt: 

In [373]:
## version2
youtuberList = []

for i in range(len(userDf)):
    driver.get(userDf.url[i])
    time.sleep(1.5)
    
    ## 1.구독자가 있는 경우 ##
    try:
        guDok = driver.find_element_by_xpath('//*[@title="구독정보"]').get_attribute('href')
        driver.get(guDok) 
        time.sleep(2)
        
        NumOfPagedowns = 20
        body = driver.find_element_by_tag_name("body")
        while NumOfPagedowns:
            body.send_keys(Keys.PAGE_DOWN)
            time.sleep(0.5)
            NumOfPagedowns -= 1
            
        youtuberEles = driver.find_elements_by_css_selector('#title')
        youtuberTexts = ', '.join([ youtuberEle.text for youtuberEle in youtuberEles ])
        youtuberList.append([userDf.이름[i], youtuberTexts])
        continue
        
    ## 2.구독자가 없는 경우 ##
    except: 
        youtuberList.append([userDf.이름[i], '없음'])
        continue

In [378]:
youtuberDf = pd.DataFrame(youtuberList, columns = ['이름', '구독자'])
youtuberDf.head()

,이름,구독자
0,pat ty,없음
1,레브,없음
2,룽이는섭섭해,없음
3,유준상,"사물궁이 잡학지식, 양선수의 온라인PT, 굿메신저, 장삐쭈, 개리형, 20대 버스기..."
4,잘생김,없음


In [379]:
youtuberDf.to_excel('더마토리_유튜버구독자정보.xlsx')

In [2]:
youtuberDf1 = pd.read_excel('더마토리_유튜버구독자정보.xlsx', index_col=0)
youtuberDf1.head()

,이름,구독자
0,pat ty,없음
1,레브,없음
2,룽이는섭섭해,없음
3,유준상,"사물궁이 잡학지식, 양선수의 온라인PT, 굿메신저, 장삐쭈, 개리형, 20대 버스기..."
4,잘생김,없음


In [3]:
youtuberDf1.tail()

,이름,구독자
3068,강영재,없음
3069,오수,없음
3070,정형주,"바이스톰 코리아, Kritter Klub, 무지막지한 막무家네, 해그린달 haegr..."
3071,sh hb,없음
3072,TheStrokes,없음


In [4]:
youtuberDfNone = youtuberDf1[youtuberDf1['구독자']!='없음']
len(youtuberDfNone)

550

In [9]:
youtuberDfNone = youtuberDfNone.reset_index(drop=True)
youtuberDfNone.head()

,이름,구독자
0,유준상,"사물궁이 잡학지식, 양선수의 온라인PT, 굿메신저, 장삐쭈, 개리형, 20대 버스기..."
1,Amy Kim,"에이프롬 ÁFROM, Yoo Jin Kim김유진 미국변호사, 주양JUYANG, 러프..."
2,배 배,"소련여자 Soviet girl in Seoul, Doenda된다, 샒의 삶Serim..."
3,안지현,"커픽처스, Yuka Kinoshita木下ゆうか, 닥터프렌즈, 소녀의행성GirlsPl..."
4,수지,"_노잼봇Nojambot, 수저격, Allure Korea, 유기농 햄치치, 티끌모아..."


## 구독자 count

In [15]:
you1 = youtuberDfNone.구독자.to_list()
you1[0]

'사물궁이 잡학지식, 양선수의 온라인PT, 굿메신저, 장삐쭈, 개리형, 20대 버스기사 이야기, 킴성태TV, 고니드럼, 백종원의 요리비책, 수상한녀석들, 디렉터 파이, 하우빌드, 비슷해보이즈, 현동명의 공사비 닷컴., 최군TV [무삭제 풀영상], 욘 바인첼, 정기사, 무빅 [Movig], 신세경 sjkuksee, encarTV 엔카티비, 티엔품[TiNPum]의 게임채널, 버블디아, Ripple_S, 창이의일상, battlegrounds sunplay, , '

In [30]:
youtuberOne = ', '.join([you for you in you1])
youtuberOne = pd.Series(youtuberOne).apply(lambda x: x.split(', '))
youtuberOne = pd.DataFrame(youtuberOne[0], columns = ['구독자'])
youtuberOne.head()

,구독자
0,사물궁이 잡학지식
1,양선수의 온라인PT
2,굿메신저
3,장삐쭈
4,개리형


In [34]:
youtuberCount = youtuberOne.구독자.value_counts().reset_index()
youtuberCount.to_excel('더마토리_유튜브구독자Count.xlsx')

In [13]:
## 댓글 단 사람들의 구독자 리스트
userList = youtuberDfNone.구독자.apply(lambda x: x.split(', '))
userList.head()

0    [사물궁이 잡학지식, 양선수의 온라인PT, 굿메신저, 장삐쭈, 개리형, 20대 버스...
1    [에이프롬 ÁFROM, Yoo Jin Kim김유진 미국변호사, 주양JUYANG, 러...
2    [소련여자 Soviet girl in Seoul, Doenda된다, 샒의 삶Seri...
3    [커픽처스, Yuka Kinoshita木下ゆうか, 닥터프렌즈, 소녀의행성GirlsP...
4    [_노잼봇Nojambot, 수저격, Allure Korea, 유기농 햄치치, 티끌모...
Name: 구독자, dtype: object

In [37]:
item_corpus = []

for i in userList :
    
    for j in range(0, 5) :
    
        item = random.sample(i, len(i))
        item_corpus.append(item)
    
print(len(item_corpus))

2750


In [39]:
# start time
start_time = time.time() 

model = Word2Vec(item_corpus,         # 리스트 형태의 데이터
                 sg=1,         # 0: CBOW, 1: Skip-gram
                 size=300,     # 벡터 크기
                 window=1,     # 고려할 앞뒤 폭(앞뒤 3단어)
                 min_count=5,  # 사용할 단어의 최소 빈도(1회 이하 단어 무시)
                 workers=2)    # 동시에 처리할 작업 수(코어 수와 비슷하게 설정)
# time check
print("\n ---------- %s seconds ----------" %(round(time.time() - start_time)))

C:\Users\Administrator\Anaconda3\lib\site-packages\gensim\models\base_any2vec.py:743: UserWarning: C extension not loaded, training will be slow. Install a C compiler and reinstall gensim for fast training.
  "C extension not loaded, training will be slow. "



 ---------- 429 seconds ----------


In [41]:
## 전체 학습에 사용된 단어의 개수
vocab = list(model.wv.vocab)
len(vocab)

12427

In [49]:
## 데이터프레임 저장
vocab = list(model.wv.vocab)
X = model[vocab]

Clustering = pd.DataFrame(X, index = vocab)
Clustering.to_csv('더마토리_유튜브_corpus5_size300_min5.csv')

C:\Users\Administrator\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  This is separate from the ipykernel package so we can avoid doing imports until


In [346]:
## 중복제거하는 코드
list(set(youtuberList[0][1].split(', ')))

['',
 '20대 버스기사 이야기',
 '킴성태TV',
 '백종원의 요리비책',
 '디렉터 파이',
 '양선수의 온라인PT',
 '굿메신저',
 '개리형',
 '수상한녀석들',
 '하우빌드',
 '장삐쭈',
 '고니드럼',
 '사물궁이 잡학지식']